#Analyzing Social Networks

The networks discussed in this recitation are **“egocentric” networks**. 

- The term **“ego”** is used to denote a person connected to everyone in the network. 
- An ego network is the social world from **ego**'s point of view. 
- It is convention to use the term **“alter”** to refer to anyone else in the network. 
- This way, one can talk about both friends and followers or fans; it does not matter what role they play: from **ego's perspective**, they are all alters.

The data used in this recitation is from https://snap.stanford.edu/data/ego-Facebook.html. The data file facebook_combined.txt is already included in the zip folder which contains this notebook.



In [ ]:
import pandas as pd    # for reading and manipulating tabular data
import networkx as nx  # for constructing and studying networks
import numpy as np     # for arrays
#import community           # for community structure later
import collections          # for manipulation tuples and zipping objects
import statistics as stats  # for generating summary statistics
import time                 # for measuring computating time
from matplotlib import pyplot as plt  # for outputting nice plots
import seaborn as sns                 # for creating even nicer plots
import urllib
import gzip
get_ipython().magic(u'matplotlib inline')

In [ ]:
data_url="https://snap.stanford.edu/data/facebook_combined.txt.gz"
facebook_data_file_compressed = urllib.request.urlopen(data_url)
facebook_data_file=gzip.open(facebook_data_file_compressed ,'rt')

In [ ]:
# Create graph from edge list stored in data file
G = nx.read_edgelist(facebook_data_file,
                     create_using = nx.Graph(), # Use Graph() instead of DiGraph() for directed vs. undirected, 
                     nodetype = int) # Do not forget to specify node information type

## Exploratory Analysis

In [ ]:
print(nx.info(G))

In [ ]:
G_nodes_list = list(G.nodes())
print(G_nodes_list[:5])

In [ ]:
G_edges_list = list(G.edges())
print(G_edges_list[:5])

### Empirical Degree Distribution

In [ ]:
<ADD CODE TO FIND DEGREE DISTRIBUTION>
plt.plot(deg, counts/sum(counts))
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Degree")
plt.ylabel("Relative Frequency")

plt.show()

### Visualization
Visualize this network; choose a representation that emphasizes the separate communities in the dataset. 
Look at [this resource](https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout) to find more options for visualizing the graph. 

In [ ]:
#Visualize the graph
pos = nx.random_layout(G) # Replace this line to generate a more intuitive visualization 
# Some options: nx.circular_layout; nx.spectral_layout; nx.spring_layout; more from given link
nx.draw_networkx_nodes(G, pos,  node_size=1)
nx.draw_networkx_edges(G, pos, alpha=0.01)
plt.show()

## Analysis

**What global characteristics of the graph would be easy to compute using networkx Python package?**

These concepts should normally be a review from the lecture!

- **Diameter** of the network (i.e., what is the maximum distance between any two nodes?)

*Note: it only makes sense to talk about the diameter of a network if every node is connected, otherwise it will be infinity.*

- **Density** of the graph (i.e., how many edges do we observe in the network, as compared to the total number of possible connections?)

- **Clustering coefficient** of the network (i.e., measure of the degree to which nodes in a graph tend to cluster together)

- **Centrality Measures** of each node of the newtork (e.g. degree centrality, betweeness centrality, etc.)

In [ ]:
def graph_characteristics(graph):
    t = time.time()
    graph_diameter = <ADD CODE TO FIND GRAPH DIAMETER>
    elapsed = time.time() - t
    print('Time elapsed to get the diameter: ', elapsed)
    
    t = time.time()
    graph_density = <ADD CODE TO FIND GRAPH DENSITY>
    elapsed = time.time() - t
    print('Time elapsed to get the density: ', elapsed)
    
    t = time.time()
    graph_avg_clustering = <ADD CODE TO FIND AVERAGE CLUSTERING COEFFICIENT>
    elapsed = time.time() - t
    print('Time elapsed to get the average clustering coefficient: ', elapsed)
    
    print("Here is a quick overview of the graph profile: diameter = " + '{:.4f}'.format(graph_diameter) \
          + ", density = " + '{:.4f}'.format(graph_density) + ", average clustering coefficient = " \
          + '{:.4f}'.format(graph_avg_clustering))
    
    return()
graph_characteristics(G)

## Community Structure 

The classical **modularity** of a partition $c:\mathcal{N} \rightarrow [N]$ is defined to be 

$$
Q \triangleq \frac{1}{2m} \sum_{u,v \in \mathcal{N}} \big( A_{uv} - \frac{k_u k_v}{2m} \big) \delta(c(u), c(v))\;.
$$

Intuitively, the modularity measures how many edges are observed *within* communities and compares that to a configuration null-model. High values of the modularity imply that there are many more edges within communities than would be expected. Many community-detection algorithms seek partitions that maximize $Q$. 

The method below uses the [*Louvain algorithm*](https://en.wikipedia.org/wiki/Louvain_Modularity) to calculate community partitions with high modularity. 

In [ ]:
!pip install python_louvain
from community import community_louvain

Use the community package [here](https://python-louvain.readthedocs.io/en/latest/api.html) to implement the Louvain algorithm, and find distinct clusters in this dataset. 

In [ ]:
<ADD CODE TO FIND THE PARTITION>
communities = [partition.get(node) for node in G.nodes()]
print('The number of communities is {}.'.format(len(np.unique(communities))))

# Let's assign each node to its given community
nx.set_node_attributes(G, partition, name='community')


In [ ]:
# No need to change this block
colors = [G.nodes()[node]['community'] for node in G.nodes()]

fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111)
ax.axis('off')

nx.draw_networkx_nodes(G, pos,  node_size=20, node_color = communities)
nx.draw_networkx_edges(G, pos, alpha=0.1)
plt.show()